<a href="https://colab.research.google.com/github/AbdurNawaz/Temporal-Difference-Methods/blob/master/Temporal_Diff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import gym
import numpy as np
import random
import math
from collections import defaultdict, deque
import matplotlib.pyplot as plt

import check_test
from plot_utils import plot_values


In [0]:
env = gym.make('CliffWalking-v0')

In [5]:
print(env.action_space)
print(env.observation_space)

Discrete(4)
Discrete(48)


In [0]:
def update_Q_sarsa(alpha, gamma, Q, state, action, reward, next_state = None, next_action = None):
  current = Q[state][action]
  Qsa_next = Q[next_state][next_action] if next_state is not None else 0
  target = reward + (gamma*Qsa_nest)
  new_value = current + alpha*(target - current)
  return new_value

def epsilon_greedy(Q, state, nA, eps):
  if random.random() > eps:
    return np.argmax(Q[state])
  else:
    return random.choice(np.arange(nA))